<img src = "bgsedsc_0.jpg">

# Category encoding examples

Let's first import required libraries and data.

In [1]:
import pandas as pd
import numpy as np
from category_encoders.binary import BinaryEncoder

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction import DictVectorizer

In [2]:
# Create some data
train= pd.DataFrame(['Spain','France','UK','France','Germany','Italy','Portugal'],columns=['Country'])
train['MyQuant']=[23,5,12,8,34,14,25]
train['Sex']=['M','F','F','M','F','M','M']

test= pd.DataFrame(['Greece','Germany','France','Germany'],columns=['Country'])
test['MyQuant']=[18,35,12,24]
test['Sex']=['F','M','F','M']

train.head()

,Country,MyQuant,Sex
0,Spain,23,M
1,France,5,F
2,UK,12,F
3,France,8,M
4,Germany,34,F


In [3]:
DummyCandidates=['Country','Sex']
NotDummies=train.columns[~train.columns.isin(DummyCandidates)]

## Option 1: get.dummies (Pandas)

In [4]:

train1=pd.get_dummies(train, prefix=DummyCandidates, 
                   columns=DummyCandidates,
                  drop_first=True)
train1.head()

,MyQuant,Country_Germany,Country_Italy,Country_Portugal,Country_Spain,Country_UK,Sex_M
0,23,0,0,0,1,0,1
1,5,0,0,0,0,0,0
2,12,0,0,0,0,1,0
3,8,0,0,0,0,0,1
4,34,1,0,0,0,0,0


In [5]:
test1=pd.get_dummies(test, prefix=DummyCandidates, 
                   columns=DummyCandidates,
                  drop_first=True)
test1.head()

,MyQuant,Country_Germany,Country_Greece,Sex_M
0,18,0,1,0
1,35,1,0,1
2,12,0,0,0
3,24,1,0,1


Be careful with your reference category if using drop_first=True.
Also with categories that differ from train to test set.

## Option 2: OneHotEncoder

In [6]:
NotDummies=train.columns[~train.columns.isin(DummyCandidates)]
my_enc2=OneHotEncoder(sparse=False, handle_unknown='ignore') #drop first not allowed
my_enc2.fit(train[DummyCandidates])
train2=my_enc2.transform(train[DummyCandidates])
train2=pd.DataFrame(train2, columns=my_enc2.get_feature_names(DummyCandidates))

# Add remaining
aux=train[NotDummies]
train2=pd.concat([train2,aux],axis=1)

train2.head()


,Country_France,Country_Germany,Country_Italy,Country_Portugal,Country_Spain,Country_UK,Sex_F,Sex_M,MyQuant
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,23
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,34


In [7]:
test2=my_enc2.transform(test[DummyCandidates])
test2=pd.DataFrame(test2, columns=my_enc2.get_feature_names(DummyCandidates))
# Add remaining
aux=test[NotDummies]
test2=pd.concat([test2,aux],axis=1)
test2

,Country_France,Country_Germany,Country_Italy,Country_Portugal,Country_Spain,Country_UK,Sex_F,Sex_M,MyQuant
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,35
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,24


You can manage differences between train an test set categories using different parameters. You can drop one category at the end or use drop='first' parameter (but then it wouldn't manage unknown category values).

## Option 3: Binary encoding

In [8]:
my_enc3 = BinaryEncoder(cols=DummyCandidates).fit(train)

In [9]:
train3= my_enc3.transform(train)
inverse=my_enc3.inverse_transform(train3) # transform back to original columns
train3['Country']=train['Country']
train3

,Country_0,Country_1,Country_2,Country_3,MyQuant,Sex_0,Sex_1,Country
0,0,0,0,1,23,0,1,Spain
1,0,0,1,0,5,1,0,France
2,0,0,1,1,12,1,0,UK
3,0,0,1,0,8,0,1,France
4,0,1,0,0,34,1,0,Germany
5,0,1,0,1,14,0,1,Italy
6,0,1,1,0,25,0,1,Portugal


In [10]:
inverse

,Country,MyQuant,Sex
0,Spain,23,M
1,France,5,F
2,UK,12,F
3,France,8,M
4,Germany,34,F
5,Italy,14,M
6,Portugal,25,M


In [11]:
test3=my_enc3.transform(test)
test3['Country']=test['Country']
test3.head()

,Country_0,Country_1,Country_2,Country_3,MyQuant,Sex_0,Sex_1,Country
0,0,0,0,0,18,1,0,Greece
1,0,1,0,0,35,0,1,Germany
2,0,0,1,0,12,1,0,France
3,0,1,0,0,24,0,1,Germany


Keeps directly non-categorical values. 

Note that for categories with high cardinality (a lot of unique values), binary encoding will help to reduce dimensionality of transformed dataset.

Here we just add 'Country' column for comparison purposes. It manages unknows ('Greece') as a 0 encoded label.